In [ ]:
import pandas as pd
import numpy as np 

# General Information

In [ ]:
loan_data_backup = pd.read_csv('E:\programming\Data science\Credit risk modelling\support files\loan_data_2007-2014\loan_data_2007_2014.csv')
loan_data = loan_data_backup.copy()
loan_data

# it will display all columns of dataset
pd.options.display.max_columns = None
loan_data

In [ ]:
loan_data['emp_title']

In [ ]:
loan_data.columns.values

In [ ]:
loan_data.head()

In [ ]:
loan_data.info()

In [ ]:
loan_data.shape

***
# **General Preprocessing**

## 1. emp_length

* In 'emp_length' it contain values of duration of loan
* dtype str, '4 years'
* We need to convert it into int '4 years' to 4

In [ ]:
loan_data['emp_length'].unique()

* We can use numpy to strip the str
* loan_data[:,'emp_length'] = np.chararray.strip(loan_data[:,'emp_length'], 'years')

In [ ]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('+ years','')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years','')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year','')
loan_data['emp_length_int'].fillna('0',inplace=True)

#### `Here we created new column name as 'emp_length_int'`

In [ ]:
loan_data.info()
loan_data['emp_length_int'].unique()

In [ ]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])
loan_data['emp_length_int'].dtype
loan_data.info()

## 2. earliest_cr_line
* dtype str , 'Jan-85'  month-year
* Convert it into int, 'Jan-85' > 85
> First we convert into datetime  format = '%b-%y

In [ ]:
loan_data['earliest_cr_line']

In [ ]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format = '%b-%y')

In [ ]:
type(loan_data['earliest_cr_line_date'][0])

In [ ]:
pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']

In [ ]:
loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']) / np.timedelta64(1, 'M')))

## 3. mths_since_earliest_cr_line

In [ ]:
loan_data['mths_since_earliest_cr_line'].describe()

In [ ]:
loan_data.loc[: , ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][loan_data['mths_since_earliest_cr_line'] < 0]

In [ ]:
loan_data['mths_since_earliest_cr_line'][loan_data['mths_since_earliest_cr_line'] < 0] = loan_data['mths_since_earliest_cr_line'].max()

In [ ]:
min(loan_data['mths_since_earliest_cr_line'])

## 4. term

* Same for term column

In [ ]:
loan_data['term'].unique()

In [ ]:
loan_data['term_int'] = loan_data['term'].replace(' 60 months', 60)
loan_data['term_int'] = loan_data['term_int'].replace(' 36 months',36)
loan_data['term_int'].unique()

## 4. issue_date

In [ ]:
loan_data['issue_d'].unique()

In [ ]:
loan_data['issue_date'] = pd.to_datetime(loan_data['issue_d'], format= '%b-%y')
loan_data['issue_date'].unique()

In [ ]:
pd.to_datetime('2017-12-01') - loan_data['issue_date']

In [ ]:
loan_data['months_issue_date'] = round(pd.to_numeric((pd.to_datetime('2017-12-01')- loan_data['issue_date'])/np.timedelta64(1,'M')))
loan_data['months_issue_date'].unique()
loan_data['months_issue_date'].describe()
loan_data.info()

***
# **Preprocessing few discrete variable**

* garde
* sub_grade
* home_ownership
* verification_status
* purpose
* addr_state
* initial_list_status

### We convert into dummy variable (0 or 1)

#### `We will do it in new dataframe and then conqiuate it to our dataset`

In [ ]:
pd.get_dummies(loan_data['grade'],dtype=int,prefix='Grade',prefix_sep=':')

#### Actual steps!

In [ ]:
loan_data_dummies = [pd.get_dummies(loan_data['grade'],dtype=int,prefix='Grade',prefix_sep=':'),
                     pd.get_dummies(loan_data['sub_grade'],dtype=int,prefix='sub_grade',prefix_sep=':'),
                     pd.get_dummies(loan_data['home_ownership'],dtype=int,prefix='home_ownership',prefix_sep=' '),
                     pd.get_dummies(loan_data['verification_status'],dtype=int,prefix='verification_status',prefix_sep=':'),
                    pd.get_dummies(loan_data['loan_status'], prefix = 'loan_status', prefix_sep = ':'),
                     pd.get_dummies(loan_data['purpose'],dtype=int,prefix='purpose',prefix_sep=':'),
                     pd.get_dummies(loan_data['addr_state'],dtype=int,prefix='addr_state',prefix_sep=':'),
                     pd.get_dummies(loan_data['initial_list_status'],dtype=int,prefix='initial_list_status',prefix_sep=':')]
loan_data_dummies

In [ ]:
loan_data_dummies

In [ ]:
loan_data_dummies = pd.concat(loan_data_dummies,axis=1)

In [ ]:
loan_data_dummies

In [ ]:
type(loan_data_dummies)

In [ ]:
loan_data = pd.concat([loan_data, loan_data_dummies],axis=1)

In [ ]:
loan_data.columns.values
loan_data.info()

***
# **Checking missing values**

In [ ]:
pd.options.display.max_rows = None
loan_data.isnull().sum()

In [ ]:
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace = True)

In [ ]:
loan_data['total_rev_hi_lim'].isnull().sum()

### So remaining missing values of columns are
* emp_title
* emp_length
* desc
* mths_since_last_delinq
* mths_since_last_record
* revol_bal
* last_pymnt_d
* next_pymnt_d
* last_credit_pull_d
* collections_12_mths_ex_med
* mths_since_last_major_derog
* annual_inc_joint
* dti_joint
* verification_status_joint
* tot_coll_amt
* tot_cur_bal
* open_acc_6m
* open_il_6m
* open_il_12m
* open_il_24m
* mths_since_rcnt_il
* total_bal_il
* il_util
* open_rv_12m
* open_rv_24m
* max_bal_bc
* all_util
* total_rev_hi_lim
* inq_fi
* total_cu_tl
* inq_last_12m
* earliest_cr_line_date
* months_since_earliest_cr_line

In [ ]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(), inplace=True)
loan_data['acc_now_delinq'].fillna(0, inplace=True)
loan_data['total_acc'].fillna(0, inplace=True)
loan_data['pub_rec'].fillna(0, inplace=True)
loan_data['open_acc'].fillna(0, inplace=True)
loan_data['inq_last_6mths'].fillna(0, inplace=True)
loan_data['delinq_2yrs'].fillna(0, inplace=True)
loan_data['emp_length_int'].fillna(0, inplace=True)
loan_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)

### So remaining missing values of columns are
* emp_title
* emp_length
* desc
* mths_since_last_delinq
* mths_since_last_record
* revol_bal
* last_pymnt_d
* next_pymnt_d
* last_credit_pull_d
* collections_12_mths_ex_med
* mths_since_last_major_derog
* annual_inc_joint
* dti_joint
* verification_status_joint
* tot_coll_amt
* tot_cur_bal
* open_acc_6m
* open_il_6m
* open_il_12m
* open_il_24m
* mths_since_rcnt_il
* total_bal_il
* il_util
* open_rv_12m
* open_rv_24m
* max_bal_bc
* all_util
* total_rev_hi_lim
* inq_fi
* total_cu_tl
* inq_last_12m
* earliest_cr_line_date
* months_since_earliest_cr_line

In [ ]:
loan_data.isnull().sum()
loan_data.info()

***
# **Model**
### * expected loss is calculated by pd, lgd and ead
* pd - Probability of default
* lgd - loss given default
* ead - Exposure at default

We use 1 for good, and 0 for bad

In [ ]:
loan_data['loan_status'].unique()

In [ ]:
loan_data['loan_status'].value_counts()/ loan_data['loan_status'].count()

In [ ]:
loan_data['good_bad'] = np.where(loan_data['loan_status'].isin(['Charged Off ', 'Default','Late (31-120 days)','Does not meet the credit policy. Status:Charged Off']),0,1)

In [ ]:
loan_data['good_bad'].unique()

In [ ]:
loan_data.info()

# There is method for turning continuous variables into categories or in descrite values


## Turning descrite values
* Method is called fine classing

### Splitting Data
Importing libraries

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
loan_data_inputs_train, loan_data_inputs_test, loan_data_targets_train, loan_data_targets_test = train_test_split(loan_data.drop('good_bad',axis=1),loan_data['good_bad'])

In [ ]:
loan_data_inputs_train.shape

In [ ]:
loan_data_targets_train.shape

In [ ]:
loan_data_inputs_test.shape

In [ ]:
loan_data_targets_test.shape

#### `We select random_state to 42 to get similar randominess in splitting of dataset everytime`

In [ ]:
loan_data_inputs_train, loan_data_inputs_test, loan_data_targets_train, loan_data_targets_test = train_test_split(loan_data.drop('good_bad', axis = 1), loan_data['good_bad'], test_size = 0.2, random_state = 42)

In [ ]:
loan_data_inputs_train.shape

In [ ]:
loan_data_targets_train.shape

In [ ]:
loan_data_inputs_test.shape

In [ ]:
loan_data_targets_test.shape

## Data Preparation : An Example

In [ ]:
# first
# df_inputs_prepr = loan_data_inputs_train
# df_targets_prepr = loan_data_targets_train
# second
df_inputs_prepr = loan_data_inputs_test
df_targets_prepr = loan_data_targets_test

### *Grade*

In [ ]:
df_inputs_prepr['grade'].unique()

In [ ]:
df1 = pd.concat([df_inputs_prepr['grade'],df_targets_prepr],axis=1)
df1.head()

In [ ]:
df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].count()

In [ ]:
df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].mean()

In [ ]:
df1 = pd.concat([df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].count(),df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].mean()], axis=1)

In [ ]:
df1

In [ ]:
df1 = df1.iloc[:, [0,1,3]]
df1

In [ ]:
df1.columns = [df1.columns.values[0], 'n_obs','prop_good']
df1

In [ ]:
df1['prop_n_obs'] = df1['n_obs']/df1['n_obs'].sum()

In [ ]:
df1

In [ ]:
df1['n_good']= df1['prop_good']*df1['n_obs']
df1['n_bad'] = (1- df1['prop_good'])*df1['n_obs']
df1

In [ ]:
df1['prop_n_good'] = df1['n_good'] / df1['n_good'].sum()
df1['prop_n_bad'] = df1['n_bad'] / df1['n_bad'].sum()

In [ ]:
df1

In [ ]:
df1['WoE'] = np.log(df1['prop_n_good']/df1['prop_n_bad'])
df1

In [ ]:

df1 = df1.sort_values(['WoE'])
df1 = df1.reset_index(drop=True)
df1

In [ ]:

df1['diff_prop_good'] = df1['prop_good'].diff().abs()
df1['diff_WoE'] = df1['WoE'].diff().abs()
df1

In [ ]:
df1['IV']= (df1['prop_n_good']-df1['prop_n_bad'])*df1['WoE']
df1['IV'] = df1['IV'].sum()
df1

## Preprocessing Discrete variables : Automating Calculations

In [ ]:
def woe_discrete(df, discrete_variabe_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variabe_name], good_bad_variable_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [ ]:
df_temp = woe_discrete(df_inputs_prepr,'grade', df_targets_prepr)
df_temp

In [ ]:
df_temp

## Visualzing the Result

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.iloc[: , 0].apply(str))
    y = df_WoE['WoE']
    plt.figure(figsize = (18, 6))
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    plt.xlabel(df_WoE.columns[0])
    plt.ylabel('Weight of Evidence')
    plt.title(str('Weight of Evidence by ' + df_WoE.columns[0]))
    plt.xticks(rotation = rotation_of_x_axis_labels)

In [ ]:
plot_by_woe(df_temp)

### *home_ownership*

for this home ownership there is no need to account other rent any none status of home_ownership separatly we can merge them into one so we will be having other_rent_any_none, own , mortgage these three variable. we decided by graph below that other has lesser value

In [ ]:
df_temp = woe_discrete(df_inputs_prepr,'home_ownership',df_targets_prepr)
df_temp

In [ ]:
df_temp['home_ownership'].value_counts()

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['home_ownership:RENT_OTHER_NONE_ANY'] = sum([df_inputs_prepr['home_ownership RENT'], df_inputs_prepr['home_ownership OTHER'],
                                                            df_inputs_prepr['home_ownership NONE'], df_inputs_prepr['home_ownership ANY']])

In [ ]:
df_inputs_prepr.columns.values

### *addr_state*

In [ ]:
df_inputs_prepr['addr_state'].sort_values().unique()

In [ ]:
df_temp = woe_discrete(df_inputs_prepr, 'addr_state', df_targets_prepr)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
if ['addr_state ND'] in df_inputs_prepr.columns.values:
    pass
else :
    print('ok')
    df_inputs_prepr['addr_state:ND'] = 0

In [ ]:
plot_by_woe(df_temp.iloc[2: -2,:])

In [ ]:
plot_by_woe(df_temp.iloc[6 : -6, :])

In [ ]:
df_inputs_prepr['addr_state'].sort_values().unique()

In [ ]:
df_inputs_prepr['addr_state:NE_IA_MS_MT_TN_RI_ND'] = sum([df_inputs_prepr['addr_state:NE'],df_inputs_prepr['addr_state:IA'], df_inputs_prepr['addr_state:MS'],df_inputs_prepr['addr_state:MT'],df_inputs_prepr['addr_state:TN'],df_inputs_prepr['addr_state:RI'],df_inputs_prepr['addr_state:ND']])
df_inputs_prepr['addr_state:NV_LA'] = sum([df_inputs_prepr['addr_state:NV'],df_inputs_prepr['addr_state:LA']])
df_inputs_prepr['addr_state:HI_SD'] = sum([df_inputs_prepr['addr_state:HI'],df_inputs_prepr['addr_state:SD'],])
df_inputs_prepr['addr_state:AZ_PA_OK'] = sum([df_inputs_prepr['addr_state:AZ'],df_inputs_prepr['addr_state:PA'],df_inputs_prepr['addr_state:OK']])
df_inputs_prepr['addr_state:MD_FL_VA_MA'] = sum([df_inputs_prepr['addr_state:MD'],df_inputs_prepr['addr_state:FL'],df_inputs_prepr['addr_state:VA'],df_inputs_prepr['addr_state:MA']])
df_inputs_prepr['addr_state:AR_DE_WV_MO_IN'] = sum([df_inputs_prepr['addr_state:AR'],df_inputs_prepr['addr_state:DE'],df_inputs_prepr['addr_state:WV'],df_inputs_prepr['addr_state:MO'],df_inputs_prepr['addr_state:IN']])
df_inputs_prepr['addr_state:OH_NC'] = sum([df_inputs_prepr['addr_state:OH'],df_inputs_prepr['addr_state:NC']])
df_inputs_prepr['addr_state:AL_MI'] = sum([df_inputs_prepr['addr_state:AL'],df_inputs_prepr['addr_state:MI']])
df_inputs_prepr['addr_state:TX_NJ_GA'] = sum([df_inputs_prepr['addr_state:TX'],df_inputs_prepr['addr_state:NJ'],df_inputs_prepr['addr_state:GA']])
df_inputs_prepr['addr_state:AK_WI'] = sum([df_inputs_prepr['addr_state:AK'],df_inputs_prepr['addr_state:WI']])
df_inputs_prepr['addr_state:WA_CT'] = sum([df_inputs_prepr['addr_state:WA'],df_inputs_prepr['addr_state:CT']])
df_inputs_prepr['addr_state:NH_KS_DC_WY_ME_ID'] = sum([df_inputs_prepr['addr_state:NH'],df_inputs_prepr['addr_state:KS'],df_inputs_prepr['addr_state:DC'],df_inputs_prepr['addr_state:WY'],df_inputs_prepr['addr_state:ID']])

In [ ]:
df_inputs_prepr.columns.values

## *verification_status*

In [ ]:
df_inputs_prepr['verification_status'].unique()

In [ ]:
df_temp = woe_discrete(df_inputs_prepr,'verification_status',df_targets_prepr)

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['initial_list_status:f'].unique()

## *purpose*

In [ ]:
df_inputs_prepr['purpose'].unique()

In [ ]:
df_temp = woe_discrete(df_inputs_prepr, 'purpose', df_targets_prepr )
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_temp['purpose'].unique()

In [ ]:
df_inputs_prepr['purpose:edu_small_house_mov'] = sum([df_inputs_prepr['purpose:educational'],df_inputs_prepr['purpose:small_business'],df_inputs_prepr['purpose:house'],df_inputs_prepr['purpose:moving']])
df_inputs_prepr['purpose:other_med'] = sum([df_inputs_prepr['purpose:other'],df_inputs_prepr['purpose:medical']])
df_inputs_prepr['purpose:debt'] = sum([df_inputs_prepr['purpose:debt_consolidation']])
df_inputs_prepr['purpose:_vaca_credit'] = sum([df_inputs_prepr['purpose:home_improvement'],df_inputs_prepr['purpose:vacation'],df_inputs_prepr['purpose:credit_card']])
df_inputs_prepr['purpose:major_rene_car_wedd'] = sum([df_inputs_prepr['purpose:major_purchase'],df_inputs_prepr['purpose:renewable_energy'],df_inputs_prepr['purpose:car'],df_inputs_prepr['purpose:wedding']])

In [ ]:
df_inputs_prepr.columns.values

## *Initail list status*

In [ ]:
# 'initial_list_status'
df_temp = woe_discrete(df_inputs_prepr, 'initial_list_status', df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)
# We plot the weight of evidence values.

## Turning continuous variable

* We use values of columns to sort it, unlike in discirte variable we use WoE.

In [ ]:
def woe_ordered_continous(df, discrete_variabe_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variabe_name], good_bad_variable_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    # df = df.sort_values(['WoE'])
    # df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

### *term_int*

In [ ]:
df_inputs_prepr['term_int'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'term_int',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

* it means 60 months are more risker than 36 months of loans (to be default)

In [ ]:
df_inputs_prepr['term_int:36'] =  np.where((df_inputs_prepr['term_int']==36),1,0)

In [ ]:
df_inputs_prepr['term_int:60'] =  np.where((df_inputs_prepr['term_int']==60),1,0)

### *emp_length_int*

In [ ]:
df_inputs_prepr['emp_length_int'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'emp_length_int',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['emp_length_int:0']= np.where(df_inputs_prepr['emp_length_int'].isin([0]),1,0)
df_inputs_prepr['emp_length_int:1']= np.where(df_inputs_prepr['emp_length_int'].isin([1]),1,0)
df_inputs_prepr['emp_length_int:2-3']= np.where(df_inputs_prepr['emp_length_int'].isin(range(2,3)),1,0)
df_inputs_prepr['emp_length_int:4-5']= np.where(df_inputs_prepr['emp_length_int'].isin(range(4,5)),1,0)
df_inputs_prepr['emp_length_int:6-8']= np.where(df_inputs_prepr['emp_length_int'].isin(range(6,8)),1,0)
df_inputs_prepr['emp_length_int:9']= np.where(df_inputs_prepr['emp_length_int'].isin([9]),1,0)
df_inputs_prepr['emp_length_int:10']= np.where(df_inputs_prepr['emp_length_int'].isin([10]),1,0)

## *months_issue_date*

In [ ]:
df_inputs_prepr['months_issue_date'].unique()

> factor is common term for catogery

In [ ]:
df_inputs_prepr['mths_since_issue_d_factor'] = pd.cut(df_inputs_prepr['months_issue_date'],50)

In [ ]:
df_inputs_prepr['mths_since_issue_d_factor']

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'mths_since_issue_d_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
plot_by_woe(df_temp.iloc[3:,:],90)

In [ ]:
df_inputs_prepr['months_issue_date:<38'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(38)),1,0)
df_inputs_prepr['months_issue_date:40-49'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(38,49)),1,0)
df_inputs_prepr['months_issue_date:49-56'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(49,56)),1,0)
df_inputs_prepr['months_issue_date:56-60'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(56,60)),1,0)
df_inputs_prepr['months_issue_date:60-66'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(60,66)),1,0)
df_inputs_prepr['months_issue_date:66-74'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(66,74)),1,0)
df_inputs_prepr['months_issue_date:74-82'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(74,82)),1,0)
df_inputs_prepr['months_issue_date:>82'] = np.where(df_inputs_prepr['months_issue_date'].isin(range(82, int(df_inputs_prepr['months_issue_date'].max()))),1,0)

## *int_rate*

In [ ]:
df_inputs_prepr['int_rate'].unique()

In [ ]:
df_inputs_prepr['int_rate_factor'] = pd.cut(df_inputs_prepr['int_rate'],50)

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'int_rate_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

> From the graph it is concluded that higher the interest, higher the probalitiy to get default

In [ ]:
df_inputs_prepr['int_rate:<7.071'] = np.where((df_inputs_prepr['int_rate']<= 7.071),1,0)
df_inputs_prepr['int_rate:7.071-11.199'] = np.where((df_inputs_prepr['int_rate']>7.071)&(df_inputs_prepr['int_rate']<11.199),1,0)
df_inputs_prepr['int_rate:11.199-13.263'] = np.where((df_inputs_prepr['int_rate']>11.199)&(df_inputs_prepr['int_rate']<13.263),1,0)
df_inputs_prepr['int_rate:13.263-15.327'] = np.where((df_inputs_prepr['int_rate']>13.263)&(df_inputs_prepr['int_rate']<15.327),1,0)
df_inputs_prepr['int_rate:15.327-19.868'] = np.where((df_inputs_prepr['int_rate']>15.327)&(df_inputs_prepr['int_rate']<19.868),1,0)
df_inputs_prepr['int_rate:19.868-25.234'] = np.where((df_inputs_prepr['int_rate']>19.868)&(df_inputs_prepr['int_rate']<25.234),1,0)
df_inputs_prepr['int_rate:>25.324'] = np.where((df_inputs_prepr['int_rate']>25.234),1,0)

## *funded_amt_factor*

In [ ]:
df_inputs_prepr['funded_amt_factor'] = pd.cut(df_inputs_prepr['funded_amnt'],50)
df_temp = woe_ordered_continous(df_inputs_prepr,'funded_amt_factor',df_targets_prepr)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp,90)

> We will not consider this variable for PD model as it is flutuating around one line or it will result as a one line

## *mths_since_earliest_cr_line*

In [ ]:
df_inputs_prepr['mths_since_earliest_cr_line'].unique()

In [ ]:
df_inputs_prepr['mths_since_earliest_cr_line_factor'] = pd.cut(df_inputs_prepr['mths_since_earliest_cr_line'],50)
df_temp = woe_ordered_continous(df_inputs_prepr,'mths_since_earliest_cr_line_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,45)

In [ ]:
plot_by_woe(df_temp.iloc[6:,:],90)

In [ ]:
df_inputs_prepr['mths_since_earliest_cr_line_factor:<126'] = np.where(df_inputs_prepr['mths_since_earliest_cr_line_factor'].isin(range(127)),1,0)
df_inputs_prepr['mths_since_earliest_cr_line_factor:126-173'] = np.where(df_inputs_prepr['mths_since_earliest_cr_line_factor'].isin(range(127,173)),1,0)
df_inputs_prepr['mths_since_earliest_cr_line_factor:173-345'] = np.where(df_inputs_prepr['mths_since_earliest_cr_line_factor'].isin(range(173,345)),1,0)
df_inputs_prepr['mths_since_earliest_cr_line_factor:345-438'] = np.where(df_inputs_prepr['mths_since_earliest_cr_line_factor'].isin(range(345,438)),1,0)
df_inputs_prepr['mths_since_earliest_cr_line_factor:>438'] = np.where(df_inputs_prepr['mths_since_earliest_cr_line_factor'].isin(range(438, int(df_inputs_prepr['mths_since_earliest_cr_line'].max()))),1,0)

## *installment*
* it as numuberous data and cannot be used for pd model so not to use for pd model

In [ ]:
df_inputs_prepr['installment_factor'] = pd.cut(df_inputs_prepr['installment'], 50)
df_temp = woe_ordered_continous(df_inputs_prepr, 'installment_factor', df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# df_inputs_prepr['installment_factor:14-127'] =np.where((df_inputs_prepr['installment_factor']>=14.276) & (df_inputs_prepr['installment_factor']<=127.216),1,0)
# df_inputs_prepr['installment_factor:127-517'] =np.where((df_inputs_prepr['installment_factor']>127.216) & (df_inputs_prepr['installment_factor']<=517.625),1,0)
# df_inputs_prepr['installment_factor:517-684'] =np.where((df_inputs_prepr['installment_factor']>517.625) & (df_inputs_prepr['installment_factor']<=584.944),1,0)
# df_inputs_prepr['installment_factor:684-1075'] =np.where((df_inputs_prepr['installment_factor']>584.944) & (df_inputs_prepr['installment_factor']<=1075.353),1,0)
# df_inputs_prepr['installment_factor:1075-1298'] =np.where((df_inputs_prepr['installment_factor']>1075.353) & (df_inputs_prepr['installment_factor']<=1298.444),1,0)
# df_inputs_prepr['installment_factor:>1298'] =np.where((df_inputs_prepr['installment_factor']>1298.444),1,0)

## *delinq_2yrs*

In [ ]:
df_inputs_prepr['delinq_2yrs'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'delinq_2yrs',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['delinq_2yrs:0'] = np.where((df_inputs_prepr['delinq_2yrs']==0),1,0)
df_inputs_prepr['delinq_2yrs:1-5'] = np.where((df_inputs_prepr['delinq_2yrs']>=1) & (df_inputs_prepr['delinq_2yrs'] <=5),1,0)
df_inputs_prepr['delinq_2yrs:>=6'] = np.where((df_inputs_prepr['delinq_2yrs']>=6),1,0)

## *inq_last_6mths*

In [ ]:
df_inputs_prepr['inq_last_6mths'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'inq_last_6mths',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['inq_last_6mths:0-3'] = np.where((df_inputs_prepr['inq_last_6mths']>=0)&(df_inputs_prepr['inq_last_6mths']<=3),1,0)
df_inputs_prepr['inq_last_6mths:3-6'] = np.where((df_inputs_prepr['inq_last_6mths']>3)&(df_inputs_prepr['inq_last_6mths']<=6),1,0)
df_inputs_prepr['inq_last_6mths:>6'] = np.where((df_inputs_prepr['inq_last_6mths']>6),1,0)

## *open_acc*

In [ ]:
df_inputs_prepr['open_acc'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'open_acc',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
plot_by_woe(df_temp.iloc[:40,:],90)

In [ ]:
df_inputs_prepr['open_acc:0'] = np.where((df_inputs_prepr['open_acc']==0),1,0)
df_inputs_prepr['open_acc:1-4'] = np.where((df_inputs_prepr['open_acc']>=1) & (df_inputs_prepr['open_acc']<=4 ),1,0)
df_inputs_prepr['open_acc:5-13'] = np.where((df_inputs_prepr['open_acc']>4) & (df_inputs_prepr['open_acc']<=13 ),1,0)
df_inputs_prepr['open_acc:13-26'] = np.where((df_inputs_prepr['open_acc']>13) & (df_inputs_prepr['open_acc']<=26 ),1,0)
df_inputs_prepr['open_acc:26-32'] = np.where((df_inputs_prepr['open_acc']>26) & (df_inputs_prepr['open_acc']<=32 ),1,0)
df_inputs_prepr['open_acc:>32'] = np.where((df_inputs_prepr['open_acc']>32),1,0)

## *pub_rec*

In [ ]:
df_inputs_prepr['pub_rec'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'pub_rec',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['pub_rec:0-2'] = np.where((df_inputs_prepr['pub_rec'] >=0) & (df_inputs_prepr['pub_rec']<=2),1,0)
df_inputs_prepr['pub_rec:2-4'] = np.where((df_inputs_prepr['pub_rec']>2) & (df_inputs_prepr['pub_rec']<=4),1,0)
df_inputs_prepr['pub_rec:>5'] = np.where((df_inputs_prepr['pub_rec']>5),1,0)

## *total_acc*

In [ ]:
df_inputs_prepr['total_acc'].unique()

In [ ]:
df_inputs_prepr['total_acc_factor'] = pd.cut(df_inputs_prepr['total_acc'],50)
df_temp = woe_ordered_continous(df_inputs_prepr,'total_acc_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_inputs_prepr['total_acc:<=27'] = np.where((df_inputs_prepr['total_acc'] <= 27), 1, 0)
df_inputs_prepr['total_acc:27-51'] = np.where((df_inputs_prepr['total_acc'] > 27) & (df_inputs_prepr['total_acc'] <= 51), 1, 0)
df_inputs_prepr['total_acc:>=51'] = np.where((df_inputs_prepr['total_acc'] > 51), 1, 0)

## *acc_now_delinq*

In [ ]:
df_inputs_prepr['acc_now_delinq'].unique()

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'acc_now_delinq',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['acc_now_delinq:0'] = np.where((df_inputs_prepr['acc_now_delinq'] == 0), 1, 0)
df_inputs_prepr['acc_now_delinq:>=1'] = np.where((df_inputs_prepr['acc_now_delinq'] >= 1), 1, 0)

## *total_rev_hi_lim_factor*

In [ ]:
df_inputs_prepr['total_rev_hi_lim'].unique()

In [ ]:
df_inputs_prepr['total_rev_hi_lim_factor'] = pd.cut(df_inputs_prepr['total_rev_hi_lim'],50)
df_temp = woe_ordered_continous(df_inputs_prepr,'total_rev_hi_lim_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs_prepr['total_rev_hi_lim:<=5K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] <= 5000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:5K-10K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 5000) & (df_inputs_prepr['total_rev_hi_lim'] <= 10000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:10K-20K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 10000) & (df_inputs_prepr['total_rev_hi_lim'] <= 20000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:20K-30K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 20000) & (df_inputs_prepr['total_rev_hi_lim'] <= 30000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:30K-40K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 30000) & (df_inputs_prepr['total_rev_hi_lim'] <= 40000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:40K-55K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 40000) & (df_inputs_prepr['total_rev_hi_lim'] <= 55000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:55K-95K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 55000) & (df_inputs_prepr['total_rev_hi_lim'] <= 95000), 1, 0)
df_inputs_prepr['total_rev_hi_lim:>95K'] = np.where((df_inputs_prepr['total_rev_hi_lim'] > 95000), 1, 0)

## *annual_inc_factor*

In [ ]:
df_inputs_prepr['annual_inc_factor'] = pd.cut(df_inputs_prepr['annual_inc'],50)

In [ ]:
df_temp = woe_ordered_continous(df_inputs_prepr,'annual_inc_factor',df_targets_prepr)
df_temp

In [ ]:
df_inputs_prepr['annual_inc_factor'] = pd.cut(df_inputs_prepr['annual_inc'],100)
df_temp = woe_ordered_continous(df_inputs_prepr,'annual_inc_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_inputs_prepr_temp = df_inputs_prepr.loc[df_inputs_prepr['annual_inc']<=140000,:]

In [ ]:
df_inputs_prepr_temp['annual_inc_factor'] = pd.cut(df_inputs_prepr_temp['annual_inc'],50)
df_temp = woe_ordered_continous(df_inputs_prepr_temp,'annual_inc_factor',df_targets_prepr[df_inputs_prepr_temp.index])
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_inputs_prepr['annual_inc:<20K'] = np.where((df_inputs_prepr['annual_inc']<=20000),1,0)
df_inputs_prepr['annual_inc:20K-30K'] = np.where((df_inputs_prepr['annual_inc']>20000) & (df_inputs_prepr['annual_inc']<=30000),1,0)
df_inputs_prepr['annual_inc:30K-40K'] = np.where((df_inputs_prepr['annual_inc']>30000) & (df_inputs_prepr['annual_inc']<=40000),1,0)
df_inputs_prepr['annual_inc:40K-50K'] = np.where((df_inputs_prepr['annual_inc']>40000) & (df_inputs_prepr['annual_inc']<=50000),1,0)
df_inputs_prepr['annual_inc:50K-60K'] = np.where((df_inputs_prepr['annual_inc']>50000) & (df_inputs_prepr['annual_inc']<=60000),1,0)
df_inputs_prepr['annual_inc:60K-70K'] = np.where((df_inputs_prepr['annual_inc']>60000) & (df_inputs_prepr['annual_inc']<=70000),1,0)
df_inputs_prepr['annual_inc:70K-80K'] = np.where((df_inputs_prepr['annual_inc']>70000) & (df_inputs_prepr['annual_inc']<=80000),1,0)
df_inputs_prepr['annual_inc:80K-90K'] = np.where((df_inputs_prepr['annual_inc']>80000) & (df_inputs_prepr['annual_inc']<=90000),1,0)
df_inputs_prepr['annual_inc:90K-100K'] = np.where((df_inputs_prepr['annual_inc']>90000) & (df_inputs_prepr['annual_inc']<=100000),1,0)
df_inputs_prepr['annual_inc:100K-120K'] = np.where((df_inputs_prepr['annual_inc']>100000) & (df_inputs_prepr['annual_inc']<=120000),1,0)
df_inputs_prepr['annual_inc:120K-140K'] = np.where((df_inputs_prepr['annual_inc']>120000) & (df_inputs_prepr['annual_inc']<=140000),1,0)
df_inputs_prepr['annual_inc:>140K'] = np.where((df_inputs_prepr['annual_inc']>140000),1,0)

### *mths_since_last_delinq*

In [ ]:
df_inputs_prepr_temp = df_inputs_prepr[pd.notnull(df_inputs_prepr['mths_since_last_delinq'])]
df_inputs_prepr_temp['mths_since_last_delinq_factor'] = pd.cut(df_inputs_prepr_temp['mths_since_last_delinq'],50)
df_temp = woe_ordered_continous(df_inputs_prepr_temp,'mths_since_last_delinq_factor',df_targets_prepr[df_inputs_prepr_temp.index])
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_inputs_prepr['mths_since_last_delinq:Missing']= np.where((df_inputs_prepr['mths_since_last_delinq'].isnull()),1,0)
df_inputs_prepr['mths_since_last_delinq:0-3'] = np.where((df_inputs_prepr['mths_since_last_delinq']>=0)&(df_inputs_prepr['mths_since_last_delinq']<=3.76),1,0)
df_inputs_prepr['mths_since_last_delinq:3-18'] = np.where((df_inputs_prepr['mths_since_last_delinq']>3.76)&(df_inputs_prepr['mths_since_last_delinq']<=18.8),1,0)
df_inputs_prepr['mths_since_last_delinq:18-30'] = np.where((df_inputs_prepr['mths_since_last_delinq']>18.8)&(df_inputs_prepr['mths_since_last_delinq']<=30.08),1,0)
df_inputs_prepr['mths_since_last_delinq:30-82'] = np.where((df_inputs_prepr['mths_since_last_delinq']>30.08)&(df_inputs_prepr['mths_since_last_delinq']<=82.72),1,0)
df_inputs_prepr['mths_since_last_delinq:>82'] = np.where((df_inputs_prepr['mths_since_last_delinq']>82.72),1,0)

## *dti_factor*

In [ ]:
df_inputs_prepr['dti_factor'] = pd.cut(df_inputs_prepr['dti'],100)
df_temp = woe_ordered_continous(df_inputs_prepr,'dti_factor',df_targets_prepr)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
df_inputs_prepr['dti:<=1.2'] = np.where((df_inputs_prepr['dti'] <= 1.2), 1, 0)
df_inputs_prepr['dti:1.2-4.3'] = np.where((df_inputs_prepr['dti'] > 1.2) & (df_inputs_prepr['dti'] <= 4.399), 1, 0)
df_inputs_prepr['dti:4.3-13.9'] = np.where((df_inputs_prepr['dti'] > 4.399) & (df_inputs_prepr['dti'] <= 13.997), 1, 0)
df_inputs_prepr['dti:13.9-21.9'] = np.where((df_inputs_prepr['dti'] > 13.997) & (df_inputs_prepr['dti'] <= 21.995), 1, 0)
df_inputs_prepr['dti:21.9-27.9'] = np.where((df_inputs_prepr['dti'] > 21.995) & (df_inputs_prepr['dti'] <= 27.993), 1, 0)
df_inputs_prepr['dti:27.9-30.7'] = np.where((df_inputs_prepr['dti'] > 27.993) & (df_inputs_prepr['dti'] <= 30.792), 1, 0)
df_inputs_prepr['dti:30.7-34.7'] = np.where((df_inputs_prepr['dti'] > 30.792) & (df_inputs_prepr['dti'] <= 34.791), 1, 0)
df_inputs_prepr['dti:>34.7'] = np.where((df_inputs_prepr['dti'] > 34.791), 1, 0)

## *mths_since_last_record*

In [ ]:
df_inputs_prepr_temp = df_inputs_prepr[pd.notnull(df_inputs_prepr['mths_since_last_record'])]
df_inputs_prepr_temp['mths_since_last_record_factor'] = pd.cut(df_inputs_prepr_temp['mths_since_last_record'], 50)
df_temp = woe_ordered_continous(df_inputs_prepr_temp, 'mths_since_last_record_factor', df_targets_prepr[df_inputs_prepr_temp.index])
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
df_inputs_prepr['mths_since_last_record:Missing'] = np.where((df_inputs_prepr['mths_since_last_record'].isnull()), 1, 0)
df_inputs_prepr['mths_since_last_record:0-2'] = np.where((df_inputs_prepr['mths_since_last_record'] >= 0) & (df_inputs_prepr['mths_since_last_record'] <= 2.42), 1, 0)
df_inputs_prepr['mths_since_last_record:2-12'] = np.where((df_inputs_prepr['mths_since_last_record'] > 2.42) & (df_inputs_prepr['mths_since_last_record'] <= 12.1), 1, 0)
df_inputs_prepr['mths_since_last_record:12-29'] = np.where((df_inputs_prepr['mths_since_last_record'] > 12.1) & (df_inputs_prepr['mths_since_last_record'] <= 29.04), 1, 0)
df_inputs_prepr['mths_since_last_record:29-33'] = np.where((df_inputs_prepr['mths_since_last_record'] > 29.04) & (df_inputs_prepr['mths_since_last_record'] <= 33.88), 1, 0)
df_inputs_prepr['mths_since_last_record:33-85'] = np.where((df_inputs_prepr['mths_since_last_record'] > 33.88) & (df_inputs_prepr['mths_since_last_record'] <= 84.7), 1, 0)
df_inputs_prepr['mths_since_last_record:>=85'] = np.where((df_inputs_prepr['mths_since_last_record'] > 84.7), 1, 0)

# Preprocessing the Test Dataset

Step for create test data set

> df_inputs_prepr = loan_data_inputs_train,
> df_targets_prepr = loan_data_targets_train
* At first we preprocess the dataset for loan_data_inputs_train
* so after preprocessing it we store that data in loan_data_inputs_train variable


> df_inputs_prepr = loan_data_inputs_test,
>df_targets_prepr = loan_data_targets_test
* then for test dataset we change the variable values of df_inputs_prepr and df_targets_prepr 
* and run it from data preparation section to until the complete the preprocessing and save the data set in loan_data_inputs_test

In [ ]:
#first
# loan_data_inputs_train = df_inputs_prepr
# second
loan_data_inputs_test = df_inputs_prepr

In [ ]:
loan_data_inputs_train.columns.value_counts

In [ ]:
loan_data_inputs_test.columns.value_counts

## *saving the data into csv files*

In [ ]:
loan_data_inputs_train.to_csv('loan_data_inputs_train.csv')
loan_data_targets_train.to_csv('loan_data_targets_train.csv')
loan_data_inputs_test.to_csv('loan_data_inputs_test.csv')
loan_data_targets_test.to_csv('loan_data_targets_test.csv')